# Analysis of Apollo Dialogue Conversations

This notebook loads conversation data from DynamoDB and analyzes interactions.

In [6]:
import boto3
import pandas as pd
from datetime import datetime
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv("../../.env")

# Initialize DynamoDB client
aws_region = os.getenv('AWS_REGION', 'eu-north-1')
endpoint_url = os.getenv('AWS_ENDPOINT_URL')
DYNAMODB_TABLE = os.getenv('DYNAMODB_TABLE', 'apollolytics_dialogues')

if endpoint_url:
    dynamodb = boto3.resource('dynamodb', region_name=aws_region, endpoint_url=endpoint_url)
else:
    dynamodb = boto3.resource('dynamodb', region_name=aws_region)

# Get the table
table = dynamodb.Table(DYNAMODB_TABLE)

# Scan the table
response = table.scan()
items = response['Items']

# Handle pagination if there are more results
while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    items.extend(response['Items'])

# Convert to DataFrame
df = pd.DataFrame(items)


# Convert timestamp to datetime - handle string timestamps
df['timestamp'] = pd.to_numeric(df['timestamp'], errors='coerce')
df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')

# Sort by session_id and timestamp
df = df.sort_values(['session_id', 'timestamp'])

# Display basic info
print(f"Total records: {len(df)}")
print(f"Unique sessions: {df['session_id'].nunique()}")
print("\nColumns in the dataset:")
print(df.columns.tolist())

# Display first few rows
df.head()

Total records: 559
Unique sessions: 105

Columns in the dataset:
['event_type', 'created_at', 'session_id', 'timestamp', 'dialogue_mode', 'origin_url', 'article', 'message_content', 'role', 'message_id', 'transcript', 'reason', 'prolific_id', 'propaganda_result', 'content', 'timing_info', 'datetime']


,event_type,created_at,session_id,timestamp,dialogue_mode,origin_url,article,message_content,role,message_id,transcript,reason,prolific_id,propaganda_result,content,timing_info,datetime
277,session_init,2025-05-20T16:34:41.017249,012e16f7-2d9a-416d-8bf9-9b8c1605bf4e,1.747759e+09,critical,http://localhost:3000/dialogue/positive,asd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-20 16:34:41
278,propaganda_analysis,2025-05-20T16:34:42.095693,012e16f7-2d9a-416d-8bf9-9b8c1605bf4e,1.747759e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'contextualization', 'data': {}, 'use...",NaN,NaN,2025-05-20 16:34:42
279,NaN,2025-05-20T16:35:16.867553,012e16f7-2d9a-416d-8bf9-9b8c1605bf4e,1.747759e+09,NaN,NaN,NaN,NaN,assistant,assistant_608c7248-7681-4072-aa0d-2af926da4e2c,NaN,NaN,NaN,NaN,What are your thoughts on the importance of sk...,"{'model_generation_time': 34.72401285171509, '...",2025-05-20 16:35:16
280,NaN,2025-05-20T16:35:36.156042,012e16f7-2d9a-416d-8bf9-9b8c1605bf4e,1.747759e+09,NaN,NaN,NaN,NaN,user,user_5825deb5-f23e-40fd-9a2b-78e62865bf6c,NaN,NaN,NaN,NaN,I think this conversation has stalled.,"{'thinking_time': 4.693, 'recording_duration':...",2025-05-20 16:35:36
156,session_init,2025-05-26T15:29:07.245837,03b6ca31-3a06-4e39-a4ab-701c3bc53dc7,1.748273e+09,positive,https://apollolytics-dialogue-kxqo4hr1x-kilian...,From welfare to Waffen: Germany’s militarism i...,NaN,NaN,NaN,NaN,NaN,asdfg,NaN,NaN,NaN,2025-05-26 15:29:07


In [2]:
df

,event_type,created_at,session_id,timestamp,dialogue_mode,origin_url,article,message_content,role,message_id,transcript,reason,prolific_id,propaganda_result,content,timing_info,datetime
231,session_init,2025-05-20T16:34:41.017249,012e16f7-2d9a-416d-8bf9-9b8c1605bf4e,1.747759e+09,critical,http://localhost:3000/dialogue/positive,asd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-20 16:34:41
232,propaganda_analysis,2025-05-20T16:34:42.095693,012e16f7-2d9a-416d-8bf9-9b8c1605bf4e,1.747759e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'contextualization', 'data': {}, 'use...",NaN,NaN,2025-05-20 16:34:42
233,NaN,2025-05-20T16:35:16.867553,012e16f7-2d9a-416d-8bf9-9b8c1605bf4e,1.747759e+09,NaN,NaN,NaN,NaN,assistant,assistant_608c7248-7681-4072-aa0d-2af926da4e2c,NaN,NaN,NaN,NaN,What are your thoughts on the importance of sk...,"{'model_generation_time': 34.72401285171509, '...",2025-05-20 16:35:16
234,NaN,2025-05-20T16:35:36.156042,012e16f7-2d9a-416d-8bf9-9b8c1605bf4e,1.747759e+09,NaN,NaN,NaN,NaN,user,user_5825deb5-f23e-40fd-9a2b-78e62865bf6c,NaN,NaN,NaN,NaN,I think this conversation has stalled.,"{'thinking_time': 4.693, 'recording_duration':...",2025-05-20 16:35:36
289,session_init,2025-05-20T16:48:58.185169,050b7eca-b404-4a85-88ea-d9de883f9c90,1.747760e+09,critical,http://localhost:3000/dialogue/positive,asd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-20 16:48:58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,session_init,2025-05-20T16:57:09.443717,fe2b56ba-6359-44d6-af96-19529ac904f5,1.747760e+09,critical,http://localhost:3000/dialogue/positive,asd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-05-20 16:57:09
268,propaganda_analysis,2025-05-20T16:57:10.631758,fe2b56ba-6359-44d6-af96-19529ac904f5,1.747760e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'contextualization', 'data': {}, 'use...",NaN,NaN,2025-05-20 16:57:10
269,NaN,2025-05-20T16:57:16.305980,fe2b56ba-6359-44d6-af96-19529ac904f5,1.747760e+09,NaN,NaN,NaN,NaN,assistant,assistant_11b5f7b6-13ee-4d16-bf52-ca17a272f767,NaN,NaN,NaN,NaN,Certainly! I noticed you provided an article a...,"{'model_generation_time': 5.625267744064331, '...",2025-05-20 16:57:16
270,NaN,2025-05-20T16:58:12.355160,fe2b56ba-6359-44d6-af96-19529ac904f5,1.747760e+09,NaN,NaN,NaN,NaN,user,user_09394b5d-bd5e-433e-aa58-568da19cde12,NaN,NaN,NaN,NaN,I think this conversation has stalled.,"{'thinking_time': 1747760286.524, 'recording_d...",2025-05-20 16:58:12


In [7]:
# Sort by created_at (newest first)
df = df.sort_values('created_at', ascending=False)
df.head()

,event_type,created_at,session_id,timestamp,dialogue_mode,origin_url,article,message_content,role,message_id,transcript,reason,prolific_id,propaganda_result,content,timing_info,datetime
346,session_end,2025-05-27T08:09:44.833366,a77096f4-f595-4e97-bedd-1e2c4049c551,1.748333e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,client_disconnected,NaN,NaN,NaN,NaN,2025-05-27 08:09:44
345,NaN,2025-05-27T08:07:02.173595,a77096f4-f595-4e97-bedd-1e2c4049c551,1.748333e+09,NaN,NaN,NaN,NaN,assistant,assistant_ff2b1c90-2aaf-4a81-8915-35ebb9674151,NaN,NaN,NaN,NaN,The article you provided takes a critical stan...,"{'model_generation_time': 9.845839023590088, '...",2025-05-27 08:07:02
344,propaganda_analysis,2025-05-27T08:06:52.315435,a77096f4-f595-4e97-bedd-1e2c4049c551,1.748333e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'contextualization', 'data': {'Poor J...",NaN,NaN,2025-05-27 08:06:52
343,session_init,2025-05-27T08:06:12.567931,a77096f4-f595-4e97-bedd-1e2c4049c551,1.748333e+09,positive,https://apollolytics-dialogue-kxqo4hr1x-kilian...,From welfare to Waffen: Germany’s militarism i...,NaN,NaN,NaN,NaN,NaN,asadcr,NaN,NaN,NaN,2025-05-27 08:06:12
171,session_end,2025-05-27T08:01:59.532754,c8e3a43d-a999-445a-bbdc-97fcd04bbf6e,1.748333e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,client_disconnected,NaN,NaN,NaN,NaN,2025-05-27 08:01:59


In [5]:
df.head(10)

,event_type,created_at,session_id,timestamp,dialogue_mode,origin_url,article,message_content,role,message_id,transcript,reason,prolific_id,propaganda_result,content,timing_info,datetime
119,session_end,2025-05-26T09:49:05.637312,dd197802-a766-45e3-895f-c2bf3c020f8a,1.748253e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conversation_stalled,NaN,NaN,NaN,NaN,2025-05-26 09:49:05
118,NaN,2025-05-26T09:49:04.461481,dd197802-a766-45e3-895f-c2bf3c020f8a,1.748253e+09,NaN,NaN,NaN,NaN,user,user_5c575326-3205-410e-befe-ca5d66bb417a,NaN,NaN,NaN,NaN,"I don't care, please stop.","{'thinking_time': 8.913, 'recording_duration':...",2025-05-26 09:49:04
117,NaN,2025-05-26T09:48:33.401023,dd197802-a766-45e3-895f-c2bf3c020f8a,1.748253e+09,NaN,NaN,NaN,NaN,assistant,assistant_b7d5db87-91fa-4ca2-a845-38aa9c00457d,NaN,NaN,NaN,NaN,Sure! Let's dive into a topic I've come across...,"{'model_generation_time': 6.959606885910034, '...",2025-05-26 09:48:33
116,propaganda_analysis,2025-05-26T09:48:26.385711,dd197802-a766-45e3-895f-c2bf3c020f8a,1.748253e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'contextualization', 'data': {}, 'use...",NaN,NaN,2025-05-26 09:48:26
115,session_init,2025-05-26T09:48:25.221982,dd197802-a766-45e3-895f-c2bf3c020f8a,1.748253e+09,positive,http://localhost:3000/dialogue/positive2,[Your positive article 2 text here],NaN,NaN,NaN,NaN,NaN,asd,NaN,NaN,NaN,2025-05-26 09:48:25
11,session_end,2025-05-26T09:44:46.876783,54aab53e-bedb-4a84-aa3b-14557c90d5e2,1.748253e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,client_disconnected,NaN,NaN,NaN,NaN,2025-05-26 09:44:46
10,propaganda_analysis,2025-05-26T09:44:41.100739,54aab53e-bedb-4a84-aa3b-14557c90d5e2,1.748253e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'contextualization', 'data': {}, 'use...",NaN,NaN,2025-05-26 09:44:41
9,session_init,2025-05-26T09:44:39.566374,54aab53e-bedb-4a84-aa3b-14557c90d5e2,1.748253e+09,critical,http://localhost:3000/dialogue/positive,sd,NaN,NaN,NaN,NaN,NaN,XXX,NaN,NaN,NaN,2025-05-26 09:44:39
375,session_end,2025-05-20T17:02:04.522638,c17c54d1-b204-423e-b024-e5178adfd9f1,1.747761e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conversation_stalled,NaN,NaN,NaN,NaN,2025-05-20 17:02:04
374,NaN,2025-05-20T17:02:03.798436,c17c54d1-b204-423e-b024-e5178adfd9f1,1.747761e+09,NaN,NaN,NaN,NaN,user,user_80dcd229-c31b-43c2-871a-edecda27ba5f,NaN,NaN,NaN,NaN,I think this conversation has stalled.,"{'thinking_time': 2.105, 'recording_duration':...",2025-05-20 17:02:03


In [18]:
df.timing_info[109]

{'model_generation_time': Decimal('11.455530881881714'),
 'model_audio_duration': Decimal('89478.48529166667'),
 'total_response_time': Decimal('89489.94082254855')}